### **Step 1**

**1. Install Required Libraries**

Here is the list of all the required libraries and their versions. Make sure the libraries are installed with the given version as multiple libraries depend on one another. If you want to install the latest version, then make sure all the required libraries are compatible with each other.

In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off
!pip -qqq install faiss-cpu==1.8.0 --progress-bar off
!pip -qqq install langchain-community==0.2.5 --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.34.2 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.44.2 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.20.3 which is incompatible.
transformers 4.44.2 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.3 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.2.41

N.B.: Here the notification `Building wheel for llama-cpp-python (pyproject.toml)` should `done` as it allows to declare which build backend used and which other dependencies are needed to build for project.

**2. Load Required Libraries**

In [2]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

In [3]:
'''Connect all the SECRET KEY, stored in the vault'''
from google.colab import userdata

### **Step 2**

**3. Set Up the File Path**

Load and set up the file path

In [4]:
file_path = "/content/12587973.pdf"

### **Step 3**

**4. Document Parsing & Spliting**

**4.1 Required Parsing function loading...**

LlamaParse is a **GenAI-native document parser** that can parse complex document data for any downstream LLM use case (RAG, agents).

It is really good at the following:

- ✅ **Broad file type support**: Parsing a variety of unstructured file types (.pdf, .pptx, .docx, .xlsx, .html) with text, tables, visual elements, weird layouts, and more.
- ✅ **Table recognition**: Parsing embedded tables accurately into text and semi-structured representations.
- ✅ **Multimodal parsing and chunking**: Extracting visual elements (images/diagrams) into structured formats and return image chunks using the latest multimodal models.
- ✅ **Custom parsing**: Input custom prompt instructions to customize the output the way you want it.

LlamaParse directly integrates with [LlamaIndex](https://github.com/run-llama/llama_index).

The free plan is up to 1000 pages a day. Paid plan is free 7k pages per week + 0.3c per additional page by default. There is a sandbox available to test the API [**https://cloud.llamaindex.ai/parse ↗**](https://cloud.llamaindex.ai/parse).

Read below for some quickstart information, or see the [full documentation](https://docs.cloud.llamaindex.ai/).

If you're a company interested in enterprise RAG solutions, and/or high volume/on-prem usage of LlamaParse, come [talk to us](https://www.llamaindex.ai/contact).

In [5]:
LLAMA_PARSE_API_KEY = userdata.get("LLAMA_PARSE_API_KEY")

parsing_instruction = f"""The provided document is resume for job application and many more.
Try to be precise while answering the questions."""

parser = LlamaParse(
    api_key = LLAMA_PARSE_API_KEY, ## LlamaParse API key
    result_type = "markdown", ## Type of parsing for showcase
    parsing_instruction = parsing_instruction, ## Additional instruction about document
)

**4.2 Load & View Document**

Loading the documents or information to be utilized through llamaparse.

In [6]:
llama_parse_documents = await parser.aload_data(file_path)
print(llama_parse_documents)

Started parsing the file under job_id f546eb35-2d6b-420a-9227-d6035b05633b
[Document(id_='a217be49-b35b-44a5-88e3-d39639ba1f6e', embedding=None, metadata={'file_path': '/content/12587973.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# SUBSTITUTE TEACHER\n\n# Summary\n\nReceiving an education on Psychology and pursuing my teaching certification has been the best professional accomplishment I have achieved. The teaching in the absence of the teacher helped me understand that I have a background to implement the strategist teach.\n\nSales Manager successful in relationship and team building. Motivates sales teams to exceed sales goals with a track record of consistently achieving employee retention, skilled in leading teams to set record-high sales figures, expand existing territories and develop new accounts. Achieve record-setting revenue gains. Talented, effective at multi-tasking and reaching sales targets. Builds loyalty and long-term 

In [7]:
parsed_doc = llama_parse_documents[0]
print(parsed_doc)

Doc ID: a217be49-b35b-44a5-88e3-d39639ba1f6e
Text: # SUBSTITUTE TEACHER  # Summary  Receiving an education on
Psychology and pursuing my teaching certification has been the best
professional accomplishment I have achieved. The teaching in the
absence of the teacher helped me understand that I have a background
to implement the strategist teach.  Sales Manager successful in
relationship and team ...


In [8]:
print(parsed_doc.id_)

a217be49-b35b-44a5-88e3-d39639ba1f6e


In [9]:
from IPython.display import Markdown
Markdown(parsed_doc.text)

# SUBSTITUTE TEACHER

# Summary

Receiving an education on Psychology and pursuing my teaching certification has been the best professional accomplishment I have achieved. The teaching in the absence of the teacher helped me understand that I have a background to implement the strategist teach.

Sales Manager successful in relationship and team building. Motivates sales teams to exceed sales goals with a track record of consistently achieving employee retention, skilled in leading teams to set record-high sales figures, expand existing territories and develop new accounts. Achieve record-setting revenue gains. Talented, effective at multi-tasking and reaching sales targets. Builds loyalty and long-term relationships with customers.

Business development executive experienced in all aspects of sales and marketing, contract negotiation and account management. Driven professional who effectively builds loyalty and long-term relationships with customers while consistently exceeding sales targets.

# Accomplishments

8620 Spectrum Center Blvd.
San Diego, CA 92123

- Attentive Sales Manager with a track record of consistently achieving employee retention and sales goals.
- 8 years in sales, customer service and personnel and payroll management.
- Driven to exceed goals and expectations.
- Talented sales professional effective at multi-tasking and reaching sales targets.
- Builds loyalty and long-term relationships with customers.

# Experience

# Substitute Teacher

01/2016 to Current
Company Name

- Manage classrooms, and help children throughout the day, by assessing the lesson of the day.
- Maintaining discipline in the classroom.

# Company Name

10/2011 to 01/2017
City, State

- Advised prospective employees on various tips and tricks that would assist them in gaining employment in the organization.
- Implement effective HR policies to ensure all practices are following labor and employment regulations.
- Increased employee retention above 90% by rigorously maintaining a positive work environment.
- Develop targeted outreach recruitment programs to recruit more minorities and meet affirmative action requirements.
- Created a website with an embedded database and FTP functionality to enable online recruitment for the organization and reducing recruitment costs by 10%.
- Develop user-friendly application forms and questionnaires to be used by the organization during staff recruitment and interviewing.
- Arbitrate labor disputes in collaboration with the legal department.

# Operations Sales Manager

09/2008 to 10/2011
Company Name
City, State

- Established operational objectives and work plans and delegated assignments to subordinate managers.
- Supervised a team of 15 area managers and 35 associates.
- Developed executive presentations and reports to facilitate project evaluation and process improvement.
- Directed planning, budgeting, vendor selection and quality assurance efforts.
- Defined clear targets and objectives and communicated them to other team members.
- Reviewed sales, customer concerns and new opportunities to drive business strategy at weekly planning sessions.
- Assessed vendor products and maintained positive vendor relations.
- Supported the sales team in writing proposals and closing contracts.
- Developed quarterly and annual sales department budgets.
- Developed a comprehensive training program for new sales associates.
- Reviewed operational records and reports to project sales and determine profitability.
- Trained all incoming sales team members.
- Maintained knowledge of current sales and promotions, policies regarding payment and exchanges and security practices.

# Insurance Agent

08/2005 to 08/2008
Fred Loya Insurance - Eagle Pass TX 78852

- Met with existing and prospective clients each week to select appropriate insurance policies.
- Calculated quotes and educated potential clients on insurance options.
- Tracked the progress of all outstanding insurance claims.
- Calculated premiums and established payment methods for sales.
- Identified and solicited sales prospects in agency databases.
- Evaluated leads obtained through direct referrals, lead databases and cold calling.
- Modeled exceptional customer service skills and appropriate diagnostic sales techniques.
- Collected all premiums on or before effective date of coverage.
- Contributed ideas and offered constructive feedback at weekly sales and training meetings.

# Education and Training

High School Diploma
2000
Eagle Pass High School
City, State

Associate of Science
---
# MEDICAL 2009 KAPLAN UNIVERSITY

# City, State

# Bachelor of Arts: Psychology minor on Child Development

# 2017 Psychology Child Development

# Languages

Fluent in English and Spanish

# Skills

- agency
- budgeting
- budgets
- business strategy
- closing
- cold calling
- interpersonal
- communication skills
- Excellent Communication
- oral
- contracts
- Critical thinking
- clients
- Client Relations
- Customer Service
- customer service skills
- databases
- database
- Fluent in English
- forms
- FTP
- HR
- Insurance
- Leadership
- legal
- Marketing
- meetings
- Mergers and acquisitions
- Works
- organizational
- policies
- executive presentations
- Problem Solver
- process improvement
- progress
- proposals
- quality assurance
- recruitment
- sales
- sales and training
- Spanish
- Strategic marketing
- supervision
- Time management
- vendor relations
- website
- written

**4.3 Load the Unstructured Data**

**4.3.1 Process 1**

We passing the document information through `UnstructuredMarkdownLoader` and storing our local system.

The `UnstructuredMarkdownLoader` is a powerful tool within the LangChain ecosystem designed to facilitate the loading of Markdown documents into a structured format suitable for downstream processing. This loader is particularly useful for developers and data scientists who work with Markdown files, allowing them to seamlessly integrate these documents into their applications.

In [10]:
# Process - 1
document_path = Path("parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [11]:
from pathlib import PosixPath
PosixPath('parsed_document.md')

PosixPath('parsed_document.md')

**4.3.2 Process 2**

This involves importing all the documents or information to be utilized.

In [12]:
# ## Process - 2
# loaded_documents = Document(page_content = parsed_doc.text,
#                             metadata={"source": "This is a resume for job application."})
# loaded_documents = [loaded_documents]

**4.4 Split the Document**

Large documents are divided into smaller pieces, for instance, sections of no more than 2000 characters each with 400 overlap.

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 400)
docs = text_splitter.split_documents(loaded_documents)

### **Step 4**

**5. Vector Embeddings and Indexing**

The data is converted into vector form using an embedding model, making it understandable for computers. For this project, `FastEmbedEmbeddings` model have been used.

In [14]:
HF_TOKEN = userdata.get("HF_TOKEN")
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

These vector embeddings are saved in a vector database, allowing them to be easily searched.

[FAISS [Facebook AI Similarity Search]](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/), a free & open-source library for efficient similarity search and clustering of dense vectors.

In [15]:
database = FAISS.from_documents(docs, embeddings)

**Retrieval:**   

When a user asks a question:


*   The user’s input is first transformed into a vector (query vector) using the same embedding model from the Data Indexing phase.
*   This query vector is then matched against all vectors in the vector database to find the most similar ones (e.g., using the Euclidean distance metric) that might contain the answer to the user’s question. This step is about identifying relevant knowledge chunks.



**6. Load Retriever**

Here, we are setting up the retriever technique with maximum top 5 similar phrase with user query would be fetch.

In [16]:
retriever = database.as_retriever(search_kwargs={"k": 5})

###**(Optional) Step 5**

**7. Apply ReRanker**

[FlashRank](https://github.com/PrithivirajDamodaran/FlashRank) is the Ultra-lite & Super-fast Python library to add re-ranking to your existing search & retrieval pipelines. It is based on SoTA cross-encoders, with gratitude to all the model owners.

In [17]:
# compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor = compressor, base_retriever = retriever
# )
compression_retriever = retriever

### **Step 6**

**8. Load Large Language Model**

Setting up the base LLM, `Llama3-70b` model via. [Groq](https://groq.com/) for retrieval process.

In [18]:
GROQ_API_KEY = userdata.get("GROQ_API_KEY")
llm = ChatGroq(temperature = 0,
               model_name = "llama3-70b-8192",
               groq_api_key = GROQ_API_KEY,
               max_tokens = 3024)

In [19]:
'''Testing the base model'''

test_llm = llm.invoke("2+2?").content
Markdown(test_llm)

The answer to 2+2 is 4.

Setting up the ground instruction as system prompt for LLM.

In [20]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(template = prompt_template,
                        input_variables = ["context", "question"])

###**(Final) Step 7**

**9. Start Question-Answering with Large Language Model for the Provided Document**

Combining all the parts i.e. LLM, retriever, system prompt for generating response for user's query.

In [21]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = compression_retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt" : prompt, "verbose" : False},
)

**9.1 Query 1**

Here we fetching the retriever with relevant query and we are getting expected response.

Watch the `query`, `response` and `source document with similarity or relevence score`.

In [22]:
query = "Tell about Operations Sales Manager role."
response = qa.invoke(str(query))
Markdown(response["result"])

**Operations Sales Manager Role**

The Operations Sales Manager role was held from 09/2008 to 10/2011 at a company in City, State. The key responsibilities in this role included:

* Establishing operational objectives and work plans, and delegating assignments to subordinate managers
* Supervising a team of 15 area managers and 35 associates
* Developing executive presentations and reports to facilitate project evaluation and process improvement
* Directing planning, budgeting, vendor selection, and quality assurance efforts
* Defining clear targets and objectives, and communicating them to other team members
* Reviewing sales, customer concerns, and new opportunities to drive business strategy at weekly planning sessions
* Assessing vendor products and maintaining positive vendor relations
* Supporting the sales team in writing proposals and closing contracts
* Developing quarterly and annual sales department budgets
* Developing a comprehensive training program for new sales associates
* Reviewing operational records and reports to project sales and determine profitability
* Training all incoming sales team members

**Additional Helpful Information**

The Operations Sales Manager role demonstrates the individual's ability to lead teams, develop strategies, and drive sales growth. The experience in this role has likely contributed to the development of strong leadership, communication, and problem-solving skills.

In [23]:
response_test = "\n"
for i, doc in enumerate(response["source_documents"]):
  response_test += str("--------------------------------\n")
  print(str(10*"--------------------------------"))
  response_test += str("##**Source Information**")+str("\t")+str(i+1)+str("\t")+str(":\n")
  print(str("Source Information")+str("\t")+str(i+1)+str("\t")+str(":"))
  response_test += str("--------------------------------\n")
  print(str(10*"--------------------------------"))
  response_test += str(doc.page_content)+str("\n")
  print(str(doc.page_content))
  response_test += str("\n\n\n")
  print("\n\n\n\n\n")

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Source Information	1	:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Maintaining discipline in the classroom.

Company Name

10/2011 to 01/2017
City, State

Advised prospective employees on various tips and tricks that would assist them in gaining employment in the organization.

Implement effective HR policies to ensure all practices are following labor and employment regulations.

Increased employee

In [24]:
Markdown(response_test)


--------------------------------
##**Source Information**	1	:
--------------------------------
Maintaining discipline in the classroom.

Company Name

10/2011 to 01/2017
City, State

Advised prospective employees on various tips and tricks that would assist them in gaining employment in the organization.

Implement effective HR policies to ensure all practices are following labor and employment regulations.

Increased employee retention above 90% by rigorously maintaining a positive work environment.

Develop targeted outreach recruitment programs to recruit more minorities and meet affirmative action requirements.

Created a website with an embedded database and FTP functionality to enable online recruitment for the organization and reducing recruitment costs by 10%.

Develop user-friendly application forms and questionnaires to be used by the organization during staff recruitment and interviewing.

Arbitrate labor disputes in collaboration with the legal department.

Operations Sales Manager

09/2008 to 10/2011
Company Name
City, State

Established operational objectives and work plans and delegated assignments to subordinate managers.

Supervised a team of 15 area managers and 35 associates.

Developed executive presentations and reports to facilitate project evaluation and process improvement.

Directed planning, budgeting, vendor selection and quality assurance efforts.

Defined clear targets and objectives and communicated them to other team members.

Reviewed sales, customer concerns and new opportunities to drive business strategy at weekly planning sessions.

Assessed vendor products and maintained positive vendor relations.

Supported the sales team in writing proposals and closing contracts.

Developed quarterly and annual sales department budgets.

Developed a comprehensive training program for new sales associates.

Reviewed operational records and reports to project sales and determine profitability.

Trained all incoming sales team members.



--------------------------------
##**Source Information**	2	:
--------------------------------
SUBSTITUTE TEACHER

Summary

Receiving an education on Psychology and pursuing my teaching certification has been the best professional accomplishment I have achieved. The teaching in the absence of the teacher helped me understand that I have a background to implement the strategist teach.

Sales Manager successful in relationship and team building. Motivates sales teams to exceed sales goals with a track record of consistently achieving employee retention, skilled in leading teams to set record-high sales figures, expand existing territories and develop new accounts. Achieve record-setting revenue gains. Talented, effective at multi-tasking and reaching sales targets. Builds loyalty and long-term relationships with customers.

Business development executive experienced in all aspects of sales and marketing, contract negotiation and account management. Driven professional who effectively builds loyalty and long-term relationships with customers while consistently exceeding sales targets.

Accomplishments

8620 Spectrum Center Blvd.
San Diego, CA 92123

Attentive Sales Manager with a track record of consistently achieving employee retention and sales goals.

8 years in sales, customer service and personnel and payroll management.

Driven to exceed goals and expectations.

Talented sales professional effective at multi-tasking and reaching sales targets.

Builds loyalty and long-term relationships with customers.

Experience

Substitute Teacher

01/2016 to Current
Company Name

Manage classrooms, and help children throughout the day, by assessing the lesson of the day.

Maintaining discipline in the classroom.

Company Name

10/2011 to 01/2017
City, State

Advised prospective employees on various tips and tricks that would assist them in gaining employment in the organization.

Implement effective HR policies to ensure all practices are following labor and employment regulations.



--------------------------------
##**Source Information**	3	:
--------------------------------
budgeting

budgets

business strategy

closing

cold calling

interpersonal

communication skills

Excellent Communication

oral

contracts

Critical thinking

clients

Client Relations

Customer Service

customer service skills

databases

database

Fluent in English

forms

FTP

HR

Insurance

Leadership

legal

Marketing

meetings

Mergers and acquisitions

Works

organizational

policies

executive presentations

Problem Solver

process improvement

progress

proposals

quality assurance

recruitment

sales

sales and training

Spanish

Strategic marketing

supervision

Time management

vendor relations

website

written



--------------------------------
##**Source Information**	4	:
--------------------------------
Assessed vendor products and maintained positive vendor relations.

Supported the sales team in writing proposals and closing contracts.

Developed quarterly and annual sales department budgets.

Developed a comprehensive training program for new sales associates.

Reviewed operational records and reports to project sales and determine profitability.

Trained all incoming sales team members.

Maintained knowledge of current sales and promotions, policies regarding payment and exchanges and security practices.

Insurance Agent

08/2005 to 08/2008
Fred Loya Insurance - Eagle Pass TX 78852

Met with existing and prospective clients each week to select appropriate insurance policies.

Calculated quotes and educated potential clients on insurance options.

Tracked the progress of all outstanding insurance claims.

Calculated premiums and established payment methods for sales.

Identified and solicited sales prospects in agency databases.

Evaluated leads obtained through direct referrals, lead databases and cold calling.

Modeled exceptional customer service skills and appropriate diagnostic sales techniques.

Collected all premiums on or before effective date of coverage.

Contributed ideas and offered constructive feedback at weekly sales and training meetings.

Education and Training

High School Diploma
2000
Eagle Pass High School
City, State

Associate of Science

MEDICAL 2009 KAPLAN UNIVERSITY

City, State

Bachelor of Arts: Psychology minor on Child Development

2017 Psychology Child Development

Languages

Fluent in English and Spanish

Skills

agency

budgeting

budgets

business strategy

closing

cold calling

interpersonal

communication skills

Excellent Communication

oral

contracts

Critical thinking

clients

Client Relations

Customer Service

customer service skills

databases

database

Fluent in English

forms

FTP

HR

Insurance

Leadership

legal

Marketing

meetings

Mergers and acquisitions

Works

organizational

policies





**9.1 Query 2**

Here we fetching the retriever with irrelevant query and we are getting expected response as this query is irrelevant for given doument.

Watch the `query`, `response` and `source document with similarity or relevence score`.

In [25]:
query = "Share the details about 'First Quarter 2024 Financial Highlights'. Share your response in table format."
response = qa.invoke(str(query))
Markdown(response["result"])

I apologize, but there is no information provided about "First Quarter 2024 Financial Highlights" in the given context. The text appears to be a resume or a CV, and it does not contain any financial data or highlights for a specific quarter.

If you meant to ask about the person's experience or skills in finance or budgeting, I can provide some information. The individual has experience in budgeting, has developed quarterly and annual sales department budgets, and has reviewed operational records and reports to project sales and determine profitability. However, there is no specific information about "First Quarter 2024 Financial Highlights".

In [26]:
response_test = "\n"
for i, doc in enumerate(response["source_documents"]):
  response_test += str("--------------------------------\n")
  print(str(10*"--------------------------------"))
  response_test += str("##**Source Information**")+str("\t")+str(i+1)+str("\t")+str(":\n")
  print(str("Source Information")+str("\t")+str(i+1)+str("\t")+str(":"))
  response_test += str("--------------------------------\n")
  print(str(10*"--------------------------------"))
  response_test += str(doc.page_content)+str("\n")
  print(str(doc.page_content))
  response_test += str("\n\n\n")
  print("\n\n\n\n\n")

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Source Information	1	:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
budgeting

budgets

business strategy

closing

cold calling

interpersonal

communication skills

Excellent Communication

oral

contracts

Critical thinking

clients

Client Relations

Customer Service

customer service skills

databases

database

Fluent in English

forms

FTP

HR

Insurance

Leadership

legal

Marketing

meetings

In [27]:
Markdown(response_test)


--------------------------------
##**Source Information**	1	:
--------------------------------
budgeting

budgets

business strategy

closing

cold calling

interpersonal

communication skills

Excellent Communication

oral

contracts

Critical thinking

clients

Client Relations

Customer Service

customer service skills

databases

database

Fluent in English

forms

FTP

HR

Insurance

Leadership

legal

Marketing

meetings

Mergers and acquisitions

Works

organizational

policies

executive presentations

Problem Solver

process improvement

progress

proposals

quality assurance

recruitment

sales

sales and training

Spanish

Strategic marketing

supervision

Time management

vendor relations

website

written



--------------------------------
##**Source Information**	2	:
--------------------------------
Maintaining discipline in the classroom.

Company Name

10/2011 to 01/2017
City, State

Advised prospective employees on various tips and tricks that would assist them in gaining employment in the organization.

Implement effective HR policies to ensure all practices are following labor and employment regulations.

Increased employee retention above 90% by rigorously maintaining a positive work environment.

Develop targeted outreach recruitment programs to recruit more minorities and meet affirmative action requirements.

Created a website with an embedded database and FTP functionality to enable online recruitment for the organization and reducing recruitment costs by 10%.

Develop user-friendly application forms and questionnaires to be used by the organization during staff recruitment and interviewing.

Arbitrate labor disputes in collaboration with the legal department.

Operations Sales Manager

09/2008 to 10/2011
Company Name
City, State

Established operational objectives and work plans and delegated assignments to subordinate managers.

Supervised a team of 15 area managers and 35 associates.

Developed executive presentations and reports to facilitate project evaluation and process improvement.

Directed planning, budgeting, vendor selection and quality assurance efforts.

Defined clear targets and objectives and communicated them to other team members.

Reviewed sales, customer concerns and new opportunities to drive business strategy at weekly planning sessions.

Assessed vendor products and maintained positive vendor relations.

Supported the sales team in writing proposals and closing contracts.

Developed quarterly and annual sales department budgets.

Developed a comprehensive training program for new sales associates.

Reviewed operational records and reports to project sales and determine profitability.

Trained all incoming sales team members.



--------------------------------
##**Source Information**	3	:
--------------------------------
SUBSTITUTE TEACHER

Summary

Receiving an education on Psychology and pursuing my teaching certification has been the best professional accomplishment I have achieved. The teaching in the absence of the teacher helped me understand that I have a background to implement the strategist teach.

Sales Manager successful in relationship and team building. Motivates sales teams to exceed sales goals with a track record of consistently achieving employee retention, skilled in leading teams to set record-high sales figures, expand existing territories and develop new accounts. Achieve record-setting revenue gains. Talented, effective at multi-tasking and reaching sales targets. Builds loyalty and long-term relationships with customers.

Business development executive experienced in all aspects of sales and marketing, contract negotiation and account management. Driven professional who effectively builds loyalty and long-term relationships with customers while consistently exceeding sales targets.

Accomplishments

8620 Spectrum Center Blvd.
San Diego, CA 92123

Attentive Sales Manager with a track record of consistently achieving employee retention and sales goals.

8 years in sales, customer service and personnel and payroll management.

Driven to exceed goals and expectations.

Talented sales professional effective at multi-tasking and reaching sales targets.

Builds loyalty and long-term relationships with customers.

Experience

Substitute Teacher

01/2016 to Current
Company Name

Manage classrooms, and help children throughout the day, by assessing the lesson of the day.

Maintaining discipline in the classroom.

Company Name

10/2011 to 01/2017
City, State

Advised prospective employees on various tips and tricks that would assist them in gaining employment in the organization.

Implement effective HR policies to ensure all practices are following labor and employment regulations.



--------------------------------
##**Source Information**	4	:
--------------------------------
Assessed vendor products and maintained positive vendor relations.

Supported the sales team in writing proposals and closing contracts.

Developed quarterly and annual sales department budgets.

Developed a comprehensive training program for new sales associates.

Reviewed operational records and reports to project sales and determine profitability.

Trained all incoming sales team members.

Maintained knowledge of current sales and promotions, policies regarding payment and exchanges and security practices.

Insurance Agent

08/2005 to 08/2008
Fred Loya Insurance - Eagle Pass TX 78852

Met with existing and prospective clients each week to select appropriate insurance policies.

Calculated quotes and educated potential clients on insurance options.

Tracked the progress of all outstanding insurance claims.

Calculated premiums and established payment methods for sales.

Identified and solicited sales prospects in agency databases.

Evaluated leads obtained through direct referrals, lead databases and cold calling.

Modeled exceptional customer service skills and appropriate diagnostic sales techniques.

Collected all premiums on or before effective date of coverage.

Contributed ideas and offered constructive feedback at weekly sales and training meetings.

Education and Training

High School Diploma
2000
Eagle Pass High School
City, State

Associate of Science

MEDICAL 2009 KAPLAN UNIVERSITY

City, State

Bachelor of Arts: Psychology minor on Child Development

2017 Psychology Child Development

Languages

Fluent in English and Spanish

Skills

agency

budgeting

budgets

business strategy

closing

cold calling

interpersonal

communication skills

Excellent Communication

oral

contracts

Critical thinking

clients

Client Relations

Customer Service

customer service skills

databases

database

Fluent in English

forms

FTP

HR

Insurance

Leadership

legal

Marketing

meetings

Mergers and acquisitions

Works

organizational

policies



